# validate results

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [19]:
import os
import argparse
from datetime import datetime

import numpy as np
import pandas as pd
from numpy.random import default_rng

from activitysim.cli import run
from activitysim.core import inject

from fru_utils import run_trip_mode_choice, comp_mode_shares

In [20]:
pd.set_option("max_columns", 500)

In [21]:
root_dir = "/mnt/c/Users/jan.zill/code/activitysim"
example_dir = os.path.join(root_dir, "test_example_mtc")

In [23]:
os.chdir(example_dir)

In [24]:
parser = argparse.ArgumentParser()
run.add_run_args(parser)
# args = parser.parse_args()
# parser.parse_args(['--sum', '7', '-1', '42'])
args = parser.parse_args(['-c', 'configs', '-o', 'output', '-d', 'data'])
#run.run(args)  # 2mins full example run
if not inject.is_injectable('preload_injectables'):
    from activitysim import abm  # register abm steps and other abm-specific injectables
run.handle_standard_args(args)  # possibly update injectables

In [25]:
# need to do this before loading checkpoint tables
trace_hh_ids = 1024353
#1024353 has trip id 642446345 (at least) where logsum and probs are different
#2821179  # has trip_ids 2464104881 and 2464104885
inject.add_injectable("trace_hh_id", trace_hh_ids)

In [26]:
%%time
trips_df_ru = run_trip_mode_choice()
trips_df = run_trip_mode_choice(choose_individual_max_utility=False)

register joint_tour_participants: no rows with household_id in [1024353].
estimation bundle trip_mode_choice not in settings file estimation.yaml


Running with %d trips 482
trip_mode_choice tour_type 'atwork' (27 trips)
trip_mode_choice tour_type 'eatout' (33 trips)
trip_mode_choice tour_type 'escort' (6 trips)
trip_mode_choice tour_type 'othdiscr' (43 trips)
trip_mode_choice tour_type 'othmaint' (46 trips)
trip_mode_choice tour_type 'school' (37 trips)
trip_mode_choice tour_type 'shopping' (77 trips)
trip_mode_choice tour_type 'social' (19 trips)
trip_mode_choice tour_type 'univ' (26 trips)
trip_mode_choice tour_type 'work' (168 trips)


register joint_tour_participants: no rows with household_id in [1024353].


Done
Running with %d trips 482


estimation bundle trip_mode_choice not in settings file estimation.yaml


trip_mode_choice tour_type 'atwork' (27 trips)
trip_mode_choice tour_type 'eatout' (33 trips)
trip_mode_choice tour_type 'escort' (6 trips)
trip_mode_choice tour_type 'othdiscr' (43 trips)
trip_mode_choice tour_type 'othmaint' (46 trips)
trip_mode_choice tour_type 'school' (37 trips)
trip_mode_choice tour_type 'shopping' (77 trips)
trip_mode_choice tour_type 'social' (19 trips)
trip_mode_choice tour_type 'univ' (26 trips)
trip_mode_choice tour_type 'work' (168 trips)
Done
CPU times: user 18.1 s, sys: 2.48 s, total: 20.6 s
Wall time: 17 s


In [27]:
c_ = ["trip_mode", "mode_choice_logsum"]
trips = trips_df_ru[c_].merge(trips_df[c_], left_index=True, right_index=True, how="outer", suffixes=["_fru", "_asim"])
pd.merge(trips_df_ru.trip_mode.value_counts(), trips_df.trip_mode.value_counts(), left_index=True, right_index=True,
         suffixes=["_fru", "_asim"])

,trip_mode_fru,trip_mode_asim
WALK,304,304
WALK_LOC,91,91
WALK_LRF,49,47
BIKE,15,17
TNC_SINGLE,6,4
DRIVEALONEFREE,5,4
WALK_HVY,5,4
SHARED2FREE,3,4
TNC_SHARED,2,4
TAXI,2,2


In [28]:
if np.allclose(trips.mode_choice_logsum_fru, trips.mode_choice_logsum_asim):
    print("Logsums agree")
else:
    ((trips.mode_choice_logsum_fru - trips.mode_choice_logsum_asim) / trips.mode_choice_logsum_fru).hist(bins=50)

Logsums agree


In [12]:
ls_ = np.abs(((trips.mode_choice_logsum_fru - trips.mode_choice_logsum_asim) / trips.mode_choice_logsum_fru))
ls_.sort_values(ascending=False)

trip_id
106170305     4.050638
106170309     0.553496
1767013721    0.482067
535694461     0.251889
943749469     0.203583
                ...   
535672921     0.000000
535620049     0.000000
535388485     0.000000
535388481     0.000000
943749471     0.000000
Length: 482, dtype: float64

In [13]:
trips.loc[trips.index == 642446345]

,trip_mode_fru,mode_choice_logsum_fru,trip_mode_asim,mode_choice_logsum_asim
trip_id,,,,
642446345,WALK_LRF,1.634462,WALK_LRF,1.593075


In [14]:
#trips.loc[np.abs(trips.mode_choice_logsum_fru - trips.mode_choice_logsum_asim) > 0]

,trip_mode_fru,mode_choice_logsum_fru,trip_mode_asim,mode_choice_logsum_asim
trip_id,,,,
86627409,WALK_LRF,2.435199,WALK_LRF,2.407248
86627413,WALK_LRF,2.665463,WALK_LRF,2.642707
86673661,WALK_LRF,1.842576,WALK_LRF,1.834538
106170305,WALK_LRF,-0.002341,WALK_LRF,-0.011825
106170309,WALK_LRF,-0.017607,WALK_LRF,-0.027353
...,...,...,...,...
1767186253,WALK_LRF,0.976319,WALK_LRF,0.880819
1767186581,WALK_HVY,1.712704,WALK_LRF,1.600736
1768237161,WALK,6.262944,WALK,6.226927


# Run MC validation

see logit._each_nest parent_nest_scale for leaf node, and simulate.compute_nested_utilities for discussion of scales
of alternatives

easiest way is to run a simulation

In [29]:
from fru_utils import get_stuff
t, ru, neu, nu, ns, nested_probs, base_probs = get_stuff()  #do_these_purposes=["social"])  #social

register joint_tour_participants: no rows with household_id in [1024353].
estimation bundle trip_mode_choice not in settings file estimation.yaml


Running with %d trips 482
trip_mode_choice tour_type 'atwork' (27 trips)
trip_mode_choice tour_type 'eatout' (33 trips)
trip_mode_choice tour_type 'escort' (6 trips)
trip_mode_choice tour_type 'othdiscr' (43 trips)
trip_mode_choice tour_type 'othmaint' (46 trips)
trip_mode_choice tour_type 'school' (37 trips)
trip_mode_choice tour_type 'shopping' (77 trips)
trip_mode_choice tour_type 'social' (19 trips)
trip_mode_choice tour_type 'univ' (26 trips)
trip_mode_choice tour_type 'work' (168 trips)
Done


In [30]:
#trips.loc[np.abs(trips.mode_choice_logsum_fru - trips.mode_choice_logsum_asim) > 0]

In [31]:
#t.loc[t.index.isin(trips.loc[np.abs(trips.mode_choice_logsum_fru - trips.mode_choice_logsum_asim) > 0].index.values)]\
#    .sort_values(by=["primary_purpose"])

In [32]:
# nest_spec = ns[0]
# for nest in logit.each_nest(nest_spec, post_order=False):
#     print(f"{nest.level}, {nest.name}, {nest.coefficient}, parent scale: {nest.parent_scale}")

In [33]:
#base_probs[7]  #615236801

In [34]:
t.loc[t.index == 642446345]

,person_id,household_id,primary_purpose,trip_num,outbound,trip_count,destination,origin,tour_id,purpose,destination_logsum,depart,trip_mode,mode_choice_logsum
trip_id,,,,,,,,,,,,,,
642446345,1958677,1024353,social,1,True,1,14,9,80305793,social,NaN,10.0,WALK_LRF,1.729057


In [183]:
num_samples = 10000
trip_id_to_check = 642446345  # 1e5 takes about 25s
#  work 615236801  also in trace log
#  social 2464104885 642446345 1767182945

bp = base_probs[7]   # 7 for social, 9 for work. Check t[t.index == trip_id_to_check] to see which purpose, and
# output above to see which index
# choose_individual_max_utility = False
#ms_comp = comp_mode_shares(bp, choose_individual_max_utility, num_samples, trip_id_to_check)
#display(ms_comp.loc[((ms_comp.mode_share_obs != 0) | (ms_comp.trip_mode != 0)), ["mode_share_obs", "trip_mode"]].T)

choose_individual_max_utility = True
ms_comp = comp_mode_shares(bp, choose_individual_max_utility, num_samples, trip_id_to_check)
display(ms_comp.loc[((ms_comp.mode_share_obs != 0) | (ms_comp.trip_mode != 0)), ["mode_share_obs", "trip_mode"]].T)

register joint_tour_participants: no rows with household_id in [1024353].
estimation bundle trip_mode_choice not in settings file estimation.yaml


Running with %d trips 482
trip_mode_choice tour_type 'social' (10000 trips)
Done


,SHARED2FREE,SHARED3FREE,TAXI,TNC_SHARED,TNC_SINGLE,WALK,WALK_LOC,WALK_LRF
mode_share_obs,0.004262,4.103657e-14,0.000084,0.00025,0.000114,0.039196,0.19705,0.759044
trip_mode,0.003800,0.000000e+00,0.000000,0.00020,0.000000,0.038000,0.20070,0.757300


# Investigate differing probs

is it due to clipping, due to idxmax always returning the first and by coincidence having equal probs somewhere, or
bad methodology?

In [121]:
def compute_probs(nested_utils, nest_spec):
    nested_exp_utils = compute_nested_probabilities(np.exp(nested_utils), nest_spec)
    base_probabilities = pd.DataFrame(index=nested_exp_utils.index)
    for nest in logit.each_nest(nest_spec, type='leaf', post_order=False):
        # skip root: it has a prob of 1 but we didn't compute a nested probability column for it
        ancestors = nest.ancestors[1:]
        base_probabilities[nest.name] = nested_exp_utils[ancestors].prod(axis=1)
    return base_probabilities

def compute_nested_probabilities(nested_exp_utilities, nest_spec):
    nested_probabilities = pd.DataFrame(index=nested_exp_utilities.index)
    for nest in logit.each_nest(nest_spec, type='node', post_order=False):
        probs = u_to_p(nested_exp_utilities[nest.alternatives])
        nested_probabilities = pd.concat([nested_probabilities, probs], axis=1)
    return nested_probabilities

EXP_UTIL_MIN = 1e-300
EXP_UTIL_MAX = np.inf

PROB_MIN = 0.0
PROB_MAX = 1.0

def u_to_p(utils, trace_label=None):
    utils_arr = utils.values
    np.clip(utils_arr, EXP_UTIL_MIN, EXP_UTIL_MAX, out=utils_arr)
    utils_arr = np.where(utils_arr == EXP_UTIL_MIN, 0.0, utils_arr)
    arr_sum = utils_arr.sum(axis=1)
    inf_utils = np.isinf(arr_sum)
    if inf_utils.any():
        print("INF")
    with np.errstate(invalid='ignore', divide='ignore'):
        np.divide(utils_arr, arr_sum.reshape(len(utils_arr), 1), out=utils_arr)
    ## if allow_zero_probs, this will cause EXP_UTIL_MIN util rows to have all zero probabilities
    utils_arr[np.isnan(utils_arr)] = PROB_MIN
    np.clip(utils_arr, PROB_MIN, PROB_MAX, out=utils_arr)
    probs = pd.DataFrame(utils_arr, columns=utils.columns, index=utils.index)
    return probs

In [177]:
from activitysim.core import logit
from numpy.random import default_rng

rng = default_rng(999)

def random_for_df(df):
    return rng.random(size=df.shape)

def add_ev1_random(df):
    uniform_rands = random_for_df(df)
    df = df + logit.inverse_ev1_cdf(uniform_rands)
    return df

def group_nests_by_level(nest_spec):
    # group nests by level, returns {level: [nest.name at that level]}
    depth = np.max([x.level for x in logit.each_nest(nest_spec)])
    nest_levels = {x: [] for x in range(1, depth+1)}
    for n in logit.each_nest(nest_spec):
        nest_levels[n.level].append(n.name)
    assert len(nest_levels[1]) == 1  # only one root
    return nest_levels

def make_choices_ru_frozen(nested_utilities, nest_spec):
    nest_utils_for_choice = add_ev1_random(nested_utilities)
    all_alternatives = list(map(lambda x: x.name, filter(lambda x: x.is_leaf, logit.each_nest(nest_spec))))

    def is_alternative(name):
        return name in all_alternatives

    nest_utils_for_choice["choice"] = None
    for level, alts in group_nests_by_level(nest_spec).items():
        if level == 1:
            continue
        no_choices_made_yet = nest_utils_for_choice["choice"].isnull()
        choice_this_level = nest_utils_for_choice.loc[no_choices_made_yet][alts].idxmax(1)
        nest_utils_for_choice.loc[no_choices_made_yet, "choice"] = \
            np.where(choice_this_level.apply(is_alternative), choice_this_level, None)

    assert not nest_utils_for_choice["choice"].isnull().any(), "No choice for XXX - implement reporting"
    choices = pd.Series(nest_utils_for_choice["choice"], index=nested_utilities.index)
    return choices, nest_utils_for_choice

In [178]:
nest_spec = ns[7]  # 7 is social
nest_util = nu[7].loc[nu[7].index == trip_id_to_check]

n = 100000

nest_util = nest_util.loc[nest_util.index.repeat(n)]
nest_util.index += np.arange(n)

display(base_probs[7].loc[base_probs[7].index == trip_id_to_check])

,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
trip_id,,,,,,,,,,,,,,,,,,,,,
642446345,0.0,0.0,0.004262,0.0,4.103657e-14,0.0,0.039196,0.0,0.19705,0.759044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000084,0.000114,0.00025


In [179]:
choices, nu_choices = make_choices_ru_frozen(nest_util, nest_spec)
choices.value_counts() / n

WALK_LRF       0.78148
WALK_LOC       0.20604
WALK           0.01194
SHARED2FREE    0.00054
Name: choice, dtype: float64

In [180]:
def group_nests_by_level(nest_spec):
    # group nests by level, returns {level: [nest]}
    depth = np.max([x.level for x in logit.each_nest(nest_spec)])
    nest_levels = {x: [] for x in range(1, depth+1)}
    for n in logit.each_nest(nest_spec):
        nest_levels[n.level].append(n)
    assert len(nest_levels[1]) == 1  # only one root
    return nest_levels

def make_choices_ru_frozen_apply(nested_utilities, nest_spec):
    nest_utils_for_choice = add_ev1_random(nested_utilities)
    all_alternatives = [nest.name for nest in logit.each_nest(nest_spec, type='leaf')]

    for level, nests_at_level in group_nests_by_level(nest_spec).items():
        nest_alts = [nest.name for nest in nests_at_level]
        if level == 1:
            assert len(nests_at_level) == 1
            assert len(nest_alts) == 1
            next_level_alts = nests_at_level[0].alternatives
            continue

        # all alternatives from the previous level
        alts_this_level = list(filter(lambda x: x in next_level_alts, nest_alts))
        #print(f"Level {level} alternatives are {alts_this_level}")
        choice_this_level = nest_utils_for_choice[nest_utils_for_choice.index.isin(alts_this_level)].idxmax()
        #print(f"choice is {choice_this_level}")
        if choice_this_level in all_alternatives:
            return choice_this_level
        chosen_nest = list(filter(lambda x: x.name == choice_this_level, nests_at_level))
        assert len(chosen_nest) == 1
        next_level_alts = chosen_nest[0].alternatives

    raise ValueError("This should never happen - no alternative found")


In [182]:
choices_apply = nest_util.apply(lambda x: make_choices_ru_frozen_apply(x, nest_spec), axis=1)
choices_apply.value_counts() / n

WALK_LRF       0.75972
WALK_LOC       0.19677
WALK           0.03901
SHARED2FREE    0.00406
TNC_SHARED     0.00025
TNC_SINGLE     0.00010
TAXI           0.00009
dtype: float64

# stuff


In [172]:
from activitysim.core import pipeline

resume_after = "trip_scheduling"
model_name = "trip_mode_choice"
chunk_size = 0  # test_mtc means no chunking

pipeline.open_pipeline(resume_after)
# preload any bulky injectables (e.g. skims) not in pipeline
inject.get_injectable('preload_injectables', None)
pipeline._PIPELINE.rng().begin_step(model_name)
#step_name = model_name
args = {}
#checkpoint = pipeline.intermediate_checkpoint(model_name)
inject.set_step_args(args)

trips = inject.get_table('trips')
tours_merged = inject.get_table('tours_merged')
network_los = inject.get_injectable('network_los')

finalise = True
if finalise:
    inject.set_step_args(None)
    #
    pipeline._PIPELINE.rng().end_step(model_name)
    pipeline.add_checkpoint(model_name)
    if not pipeline.intermediate_checkpoint():
        pipeline.add_checkpoint(pipeline.FINAL_CHECKPOINT_NAME)

    pipeline.close_pipeline()

register joint_tour_participants: no rows with household_id in [982875].


In [176]:
t_ = trips.to_frame()

In [177]:
t_.loc[t_.index == 2464104885]

,person_id,household_id,primary_purpose,trip_num,outbound,trip_count,destination,origin,tour_id,purpose,destination_logsum,depart
trip_id,,,,,,,,,,,,
2464104885,7512514,2821179,social,1,False,1,8,9,308013110,home,NaN,16.0
